## 2. 生成式大语言模型
### 2.1 简介
在当今人工智能领域，生成式大语言模型（Large Language Models，LLMs）正以前所未有的影响力改变着我们的生活和工作方式。它们像拥有无尽知识的智者，能够根据我们提出的各种问题，生成流畅、富有逻辑且富有创意的文字回答。从撰写文章、生成代码，到进行语言翻译、提供生活建议，这些模型的应用场景广泛且多样。它们背后蕴含着复杂而精妙的技术原理，而这一切都始于一个关键的架构——Transformer。接下来，我们将深入探索生成式大语言模型的世界，了解其背后的原理、训练方式以及使用范式，带你揭开其神秘的面纱。

**本章重点内容**： 生成式大语言模型的自回归预测
<!-- <img src="assets/cp_2_arch.png" alt="大语言模型概览" width="1000" height="400" title="图1. 大语言模型概览"> -->


### 2.2 大语言模型概览
#### 2.2.1 基本原理与发展
生成式大语言模型的核心架构是Transformer。想象一下，你有一张巨大的拼图，例如下图。每一块拼图都包含着一些信息。Transformer架构就像是一个超级高效的拼图大师，它能够同时查看所有拼图块，并理解不同拼图块之间的关系。这种架构通过一种称为“自注意力机制”的方式，让模型能够关注到输入文本中的重要部分，并理解它们之间的相互关系。例如，当我们输入“猫在沙发上睡觉”，模型会理解“猫”和“睡觉”之间的关系，以及“沙发”跟“睡觉”的关联性。

<img src="assets/cp2_attn.png" alt="注意力机制" width="500" height="400" title="注意力机制">


而预训练思想则是让模型在海量的文本数据中学习语言的规律和模式。这就像是让一个孩子在阅读了无数的书籍后，逐渐掌握了语言的表达方式。模型在预训练阶段会学习如何预测下一个词，通过不断地练习，它能够逐渐理解语言的结构和语义。例如，当我们给模型输入“我每天早上都会喝一杯”，它会根据之前学到的知识，预测下一个词可能是“咖啡”或“牛奶”。

随着技术的发展，生成式大语言模型的规模越来越大。这些模型的参数数量从最初的几亿个，发展到如今的几百亿甚至上千亿个。参数规模的增加就像是给模型提供了更多的“神经元”，让它能够更细致地理解语言的复杂性。例如，一个小型模型可能只能理解简单的句子结构，而一个大型模型则能够处理复杂的长句和多义词。


#### 2.2.2 主流大语言模型及参数规模
目前，市场上有多种主流的生成式大语言模型，这里我们简单列举不同大语言模型的规模，发布机构，是否开源已经发布时间。



| 模型名称          | 发布机构       | 参数规模      | 发布时间   | 开源  |
|-------------------|---------------|--------------|------------|-------|
| GPT-4o           | OpenAI        | ~1.8T*       | 2024-05    | 否    |
| GPT-4            | OpenAI        | ~1T*         | 2023-03    | 否    |
| Claude 3 Opus    | Anthropic     | ~1T*         | 2024-03    | 否    |
| Gemini 1.5 Pro   | Google        | ~1T*         | 2024-02    | 否    |
| Llama 3-70B      | Meta          | 70B          | 2024-04    | 是    |
| Qwen2-72B        | Alibaba       | 72B          | 2024-06    | 是    |
| Mixtral 8x22B    | Mistral AI    | 141B (MoE)*  | 2024-04    | 是    |
| Falcon-180B      | TII           | 180B         | 2023-09    | 是    |
| BLOOM-176B       | BigScience    | 176B         | 2022-07    | 是    |
| Command R+       | Cohere        | ~100B*       | 2024-04    | 否    |
| DeepSeek-V3      | DeepSeek      | ~128B*       | 2024-06    | 否    |
| Yi-34B           | 01.AI         | 34B          | 2023-11    | 是    |
| OLMo-65B         | Allen AI      | 65B          | 2024-02    | 是    |
| Mistral-7B       | Mistral AI    | 7B           | 2023-09    | 是    |

其中，`*` 表示未官方确认的推测值，`(MoE)` 表示混合专家模型。



### 2.3 文本生成原理与特点
#### 2.3.1 词汇表和Token
在生成式大语言模型中，词汇表就像是一个巨大的工具箱，里面包含了所有可能使用的词汇。模型在生成文本时，会从这个工具箱中选择合适的词汇来构建句子。而Token则是词汇表中的最小单位，也是语言模型可以理解并生成的最小单元，它可以是一个单词、一个标点符号，甚至是一个词的一部分。

这里我们截取了下面即将用到的Qwen2.5-Instruct的部分词汇表（完整词汇表可见模型的`vocab.json`文件）。
```
"!": 0,
"\"": 1, 
"#": 2,
"$": 3, 
"%": 4, 
"&": 5,
"'": 6, 
"(": 7, 
")": 8, 
"*": 9,
......
```
可以看到，token “!”在词汇表中的序号是0，“\”序号是1，因此在一句话中，被单独切分（tokenize）出来的“!”和“\”会被它们在词汇表中的编号0和1替代。


#### 2.3.2 自回归预测
生成式大语言模型的文本生成机制就像是在写一封长信。模型从一个已知的开头开始，比如“亲爱的”，然后根据之前的内容，预测下一个词。它会不断地重复这个过程，直到生成完整的文本。这个过程是基于概率的，模型会根据之前学到的语言模式，计算出下一个词的可能性。例如，当我们输入“今天天气很好，适合”，模型会根据上下文，预测下一个词可能是“出去玩”或“散步”。这种自回归的方式使得生成的文本具有连贯性和逻辑性。

假设我们有一个输入文本序列 $ x = (x_1, x_2, \dots, x_T) $，其中 $ x_t $ 表示第 $ t $ 个词或token。生成式大语言模型的目标是生成下一个词 $ x_{T+1} $。

模型通过计算条件概率 $ P(x_{T+1} | x_1, x_2, \dots, x_T) $ 来预测下一个词。具体来说，模型会根据已有的词序列 $ x_1, x_2, \dots, x_T $ 来计算下一个词的概率分布，然后选择概率最高的词作为预测结果。

$$
P(x_{T+1} | x_1, x_2, \dots, x_T) = \text{softmax}(f(x_1, x_2, \dots, x_T))
$$

其中，$ f $ 是模型的内部函数，通常是一个复杂的神经网络，如Transformer架构。softmax函数将模型的输出转换为概率分布，确保所有词的概率之和为1。

举一个具体的例子，我们有一个输入文本“今天天气很好，适合”，模型会根据这个输入来预测下一个词。具体步骤如下：

1. **输入序列**：$ x = (\text{今天}, \text{天气}, \text{很好}, \text{，}, \text{适合}) $
2. **计算条件概率**：模型计算 $ P(x_{T+1} | \text{今天}, \text{天气}, \text{很好}, \text{，}, \text{适合}) $
3. **选择下一个词**：模型从概率分布中选择概率最高的词，假设是“出去玩”

最终，模型生成的文本为“今天天气很好，适合出去玩”。

公式的具体含义如下：

- $ P(x_{T+1} | x_1, x_2, \dots, x_T) $：表示在给定前面所有词 $ x_1, x_2, \dots, x_T $ 的条件下，下一个词 $ x_{T+1} $ 的概率。
- $ \text{softmax}(f(x_1, x_2, \dots, x_T)) $：表示模型通过内部函数 $ f $ 计算输出，然后通过softmax函数将其转换为概率分布。


下图我们演示生成式语言模型如何根据输入“今天天气很好，适合”，来通过自回归一步生成一个token的方式进行预测的。

<img src="assets/cp2_autogressive.png" alt="自回归预测" width="500" height="600" title="图2. 自回归预测">

如图所示，最终预测结果为“出去玩”，此处“token选择”允许我们对语言模型的输出进行干预，来控制生成的风格和质量，我们将在`2.5`节中进行介绍。


#### 2.3.3 语言模型特性
一个语言模型的构建可以简单分为两个阶段，预训练（知识获取）→ 推理（知识应用）：

- 预训练：模型在海量文本上通过自监督学习（如掩码语言建模）掌握语言统计规律，构建对词汇、语法和世界知识的理解。

- 推理：基于预训练获得的知识，模型根据输入提示实时生成文本，此过程依赖自回归预测机制。

<img src="assets/cp2_tune_predict.png" alt="语言模型特性与机制的联系" width="500" height="300" title="图3. 语言模型特性与机制的联系">

预训练和推理机制赋予模型很多独特的特点：

- 首先，它们具有很强的***创意性***。模型可以根据用户的提示，生成各种新颖的想法和表达方式。例如，当我们要求模型创作一首关于“月亮”的诗歌，它可以写出“月亮在夜空中轻轻摇曳，洒下银色的光辉，照亮了寂静的大地”。这种创意性让模型在创意写作、广告文案等领域具有很大的应用价值。

- 其次，生成式模型还具有一定的***随机性***。由于其基于概率的生成机制，每次生成的结果可能会有所不同。这就像抛硬币一样，虽然我们知道正面和反面的概率，但每次的结果仍然是随机的。这种随机性使得模型能够生成多样化的文本，但也可能导致一些不可预测的结果。

- 然而，生成式模型也存在一些***局限性***。例如，它们的上下文窗口是有限的。这意味着模型在生成文本时，只能考虑一定范围内的上下文信息。如果文本过长，模型可能会丢失一些重要的信息，从而导致生成的文本出现逻辑不连贯的情况。此外，模型还可能出现“幻觉”现象，即生成一些与事实不符的内容。这是因为模型只是根据语言模式进行预测，并没有真正理解语义。例如，模型可能会错误地生成“太阳从西方升起”的句子。

总之，生成式大语言模型是一种非常强大的工具，它能够根据输入的提示生成各种文本。通过理解其文本生成原理和特点，我们可以更好地利用它来完成各种任务，同时也要注意其可能存在的局限性。


## 2.4 开源模型的使用

### 2.4.1 环境准备与依赖安装

在使用开源大语言模型之前，首先需要搭建合适的开发环境并安装所需的依赖库。以下是详细的环境准备步骤：

1. **Python 环境**：确保你的系统已经安装了 Python 3.7 或更高版本。可以通过访问 [Python 官网](https://www.python.org/) 下载并安装。

2. **PyTorch 或 TensorFlow**：根据你的硬件环境（CPU 或 GPU）选择合适的深度学习框架。这里我们以 PyTorch 为例，它提供了良好的 GPU 加速支持。

   - **CPU 版本**：

In [ ]:
! pip install torch

   - **GPU 版本**（如果你的系统有 NVIDIA GPU 并安装了 CUDA）：

In [ ]:
! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

安装完成后，可以通过以下代码验证安装是否成功：

In [1]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"Is CUDA available: {torch.cuda.is_available()}")

PyTorch version: 2.6.0
Is CUDA available: True


如果 CUDA 可用，说明你的 GPU 环境已经配置成功，可以加速模型推理。

### 2.4.2 开源模型下载与加载

加载开源大语言模型是进行文本生成的第一步。我们将使用 Qwen2.5-Insturct 模型来演示这一过程。

1. **选择模型**：在 Hugging Face 的 [Model Hub](https://huggingface.co/models) 上搜索并选择适合的模型。Qwen2.5-Insturct 是一个适合指令微调的模型，适合用于对话和问答任务。

2. **加载模型和分词器**：

首先进行transformers的安装：

In [ ]:
! pip install transformers

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# 指定模型名称或本地路径
model_name = "/root/models/Qwen2.5-0.5B-Instruct"

# 加载分词器和模型
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# 将模型移动到 GPU（如果可用）
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

print(f"Model loaded on {device}")

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model loaded on cuda


3. **模型参数说明**：
   - `AutoTokenizer`：用于将文本转换为模型可接受的输入格式（token ids）。
   - `AutoModelForCausalLM`：用于加载预训练的因果语言模型，适合文本生成任务。

加载模型后，你可以查看模型的参数量和结构：

In [3]:
# 查看模型参数量
total_params = sum(p.numel() for p in model.parameters())
print(f"Total model parameters: {total_params / 1e6:.2f}M")

# 查看模型结构
print(model)

Total model parameters: 494.03M
Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )


### 2.4.3 文字续写任务

文字续写是生成式大语言模型的基本功能之一。以下是详细的代码示例：

In [4]:
from transformers import pipeline

# 创建文本生成管道
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=device  # 指定设备
)

# 输入提示文本
prompt = "从前有一个小村庄，村里住着一位聪明的老人。"

# 生成续写文本
generated_text = text_generator(
    prompt,
    max_length=200,  # 最大生成长度
    min_length=50,   # 最小生成长度
    do_sample=True,  # 是否使用采样
    early_stopping=True  # 提前停止生成
)[0]['generated_text']

print("生成的文本：")
print(generated_text)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:638: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


生成的文本：
从前有一个小村庄，村里住着一位聪明的老人。有一天，他决定去外面看看世界。他走出了村口，看到一条大河，河水哗哗地流过。他感到非常惊讶和兴奋。
于是，他沿着河岸走了一段路，然后转了个弯，又走了好几里路。当他走到河边时，发现河对岸有一座小岛，小岛上长满了各种各样的植物。
他想：如果我每天都去这个小岛散步，会不会有好多有趣的事情发生呢？
第二天，他再次来到河边，发现河面上还漂浮着一些垃圾，而小岛上的一些植物却依然健康茁壮。
他又想：如果我每天都去这个小岛散步，会有什么有趣的经历呢？他会发现小岛上有许多有趣的东西，比如：海螺、螃蟹、贝壳等海洋生物；还有那些被风吹起的小花、小草等等。
他还想：如果我每天都去这个


## 2.5 生成过程的干预

在生成式大语言模型的生成过程中，可以通过调整参数来干预生成结果的风格和质量。这些参数主要包括温度（Temperature）和采样策略（如贪心采样、Top-k 采样、Top-p 采样）。以下是对这些参数的详细介绍以及如何通过改变这些参数来控制输出。

1. **温度（Temperature）**：
   - **高温值（如 1.0）**：增加生成结果的多样性，但可能导致结果不稳定。
   - **低温值（如 0.1）**：生成结果更稳定，但可能缺乏创造性。

   温度参数控制生成结果的随机性。具体来说，温度参数 $ T $ 影响概率分布的“尖锐度”。温度越高，概率分布越平滑，生成结果的多样性越高；温度越低，概率分布越尖锐，生成结果越稳定。

   $
   P(x_{T+1} | x_1, x_2, \dots, x_T) = \frac{\exp\left(\frac{f(x_1, x_2, \dots, x_T)}{T}\right)}{\sum_{i} \exp\left(\frac{f(x_1, x_2, \dots, x_T)}{T}\right)}
   $

   其中，$ f(x_1, x_2, \dots, x_T) $ 是模型的内部函数，通常是一个复杂的神经网络，如 Transformer 架构。$ T $ 是温度参数，控制概率分布的平滑程度。

2. **采样策略**：
   - **贪心采样（Greedy Sampling）**：始终选择概率最高的下一个词，生成结果稳定但可能单调。

     $
     x_{T+1} = \arg\max P(x_{T+1} | x_1, x_2, \dots, x_T)
     $

   - **Top-k 采样**：从概率最高的 k 个词中随机选择下一个词，平衡稳定性和多样性。

     $
     S_k = \{ x_i \mid P(x_i | x_1, x_2, \dots, x_T) \text{ 是前 } k \text{ 高的} \}
     $

     $
     x_{T+1} \sim P(x_{T+1} | x_1, x_2, \dots, x_T) \text{ 在 } S_k \text{ 中随机选择}
     $

   - **Top-p 采样（Nucleus Sampling）**：从概率累积达到 p 的最小词集中随机选择下一个词，动态调整采样范围。

     $
     S_p = \{ x_i \mid \sum_{j=1}^{i} P(x_j | x_1, x_2, \dots, x_T) \leq p \}
     $

     $
     x_{T+1} \sim P(x_{T+1} | x_1, x_2, \dots, x_T) \text{ 在 } S_p \text{ 中随机选择}
     $

通过上面的公式，我们可以得出如何通过改变这些参数控制输出：

- **温度参数 $ T $**：
  - **高温值（如 1.0）**：增加生成结果的多样性，适合需要创意和探索的场景。
  - **低温值（如 0.1）**：生成结果更稳定，适合需要准确和一致性的场景。

- **采样策略**：
  - **贪心采样**：适合需要稳定和一致性的场景，如问答系统。
  - **Top-k 采样**：适合需要平衡稳定性和多样性的场景，如创意写作。
  - **Top-p 采样**：适合需要动态调整多样性的场景，如对话系统。

### 示例代码

In [29]:
# 输入提示文本
prompt =  "请介绍一下大语言模型的注意力机制。"

# 生成严谨回答（低温值，贪心采样）
rigorous_text = text_generator(
    prompt,
    max_length=200,
    temperature=0.1,  # 低温值
    top_k=10,          # Top-k 采样，低k值
    top_p=0.5,        # Top-p 采样， 低p值
    num_return_sequences=1,
    repetition_penalty=1.2  # 避免重复
)[0]['generated_text']

print("-"*50)

print("严谨回答：")
print(rigorous_text[len(prompt):])  # 去掉提示文本部分

# 生成创意回答（高温值，Top-p 采样）
creative_text = text_generator(
    prompt,
    max_length=200,
    temperature=1.5,  # 高温值
    top_k=50,         # Top-k 采样， 高k值
    top_p=0.9,        # Top-p 采样，高p值
    num_return_sequences=1,
    repetition_penalty=1.0  # 允许一定重复
)[0]['generated_text'][len(prompt):]  # 去掉提示文本部分

print("\n创意回答：")
print(creative_text)

--------------------------------------------------
严谨回答：
 大规模预训练的语言模型，如GPT、BERT等，在处理文本时会使用一种称为“注意力机制”的技术来增强其性能和理解能力。

在传统的自然语言处理中，我们通常关注的是如何让计算机理解和生成人类语言的能力最大化。然而，随着深度学习的发展，人们发现通过引入注意力机制可以显著提高机器对输入信息的理解能力和表达质量。

### 注意力机制的基本概念

1. **注意力过程**：这是指一个特定任务（例如识别某个单词或句子）与另一个部分之间的交互作用。
2. **权重分配**：每个位置上的词被赋予不同的权重，这些权重反映了该位置上词汇的重要性。
3. **局部视图**：在一个给定的位置上，所有可能影响当前上下文的信息都被考虑进来，并且它们会被加权并进行组合以形成最终的结果。

### 为什么需要注意力机制？

- **提升理解效率**:

创意回答：
 在现实世界中，我们并不能制造语言或使用模型模拟人类的能力来完成语言工作。尽管人工智能有许多方法来理解，解读、解释和生成文本，这些都是从人类语言中“复制”下来，并不是直接创建语言或复制人的思维过程。
但是，如果我们说大语言模型（例如Transformer、Rasa、BERT等）使用到的语言模型具有某些类似于注意力机制的现象，可能有一些合理且普遍的观点。语言模型和计算机中的各种注意力机制有一些相似性，包括如何在模型输出结果之间识别并连接关键上下文信息。

然而，语言模型的任务是识别生成文本的关键句关系（如句子段落、句子成分间的联系），而不是对内容进行理解，而理解和生成的内容是一个更复杂的复杂过程。
所以，大语言模型并不能真正的具有“注意力”来解释人类和人类的理解或创造力。理解人类的能力在某种程度上仍然依赖于人类和人类之间的交互，而


可以看到，严谨回答分点清晰，显得专业可信；创意回答更偏向平铺直叙，显得通俗易懂。

## 2.6 本地化智能体

基于开源模型，可以构建本地化的智能体，用于单轮或多轮对话。

### 2.6.1 单轮对话：问答助手

单轮对话系统用于回答用户提出的问题，适合简单的问答场景。

In [30]:
def single_turn_qa(question):
    # 输入问题
    prompt = f"问题：{question}\n回答："
    
    # 生成回答
    response = text_generator(
        prompt,
        max_length=300,
        temperature=0.5,  # 中等温度值
        top_k=50,
        top_p=0.9,
        num_return_sequences=1,
        repetition_penalty=1.1
    )[0]['generated_text'].split("回答：")[1]
    
    return response

# 测试单轮问答
question = "什么是人工智能？"
answer = single_turn_qa(question)
print(f"问题：{question}")
print(f"回答：{answer}")

问题：什么是人工智能？
回答：人工智能（Artificial Intelligence，简称AI）是指使机器能够执行通常需要人类智能的任务的计算机科学分支。它包括了机器学习、自然语言处理、计算机视觉等技术领域，并且旨在让机器能够像人一样思考和解决问题。

人工智能的应用范围非常广泛，从自动驾驶汽车到语音助手，从医疗诊断系统到金融投资决策，都有许多实际应用案例。随着技术的进步，人工智能将越来越深入地融入我们的生活，改变我们的工作方式和社会结构。然而，人工智能也面临着一些挑战，如隐私保护、就业影响以及伦理道德等问题。因此，在发展人工智能的同时，我们也需要关注这些问题并寻求解决方案。


### 2.6.2 多轮对话：聊天机器人

多轮对话系统能够维护对话历史，使对话更具连贯性和上下文感知能力。

In [49]:
class MultiTurnChatbot:
    def __init__(self):
        self.history = []  # 存储对话历史
    
    def generate_response(self, user_input):
        # 构建包含历史的提示
        prompt = "对话历史：\n"
        for turn in self.history:
            prompt += f"用户：{turn['user']}\n助手：{turn['assistant']}\n"
        prompt += f"用户：{user_input}\n助手："
        
        # 生成回答
        response = text_generator(
            prompt,
            max_length=300,
            temperature=0.5,
            top_k=50,
            top_p=0.9,
            num_return_sequences=1,
            repetition_penalty=1.1
        )[0]['generated_text'].split("助手：")[-1]
        
        # 保存当前对话到历史
        self.history.append({"user": user_input, "assistant": response})
        
        return response

# 测试多轮对话
chatbot = MultiTurnChatbot()

# 第一轮对话
user_input = "什么是人工智能。"
response = chatbot.generate_response(user_input)
print("----------------对话1----------------")
print(f"用户：{user_input}")
print(f"助手：{response}")

# 第二轮对话
user_input = "列举它的应用场景。"
response = chatbot.generate_response(user_input)
print("----------------对话2----------------")
print(f"用户：{user_input}")
print(f"助手：{response}")

# 第三轮对话
user_input = "使用人工智能需要注意什么。"
response = chatbot.generate_response(user_input)
print("----------------对话3----------------")
print(f"用户：{user_input}")
print(f"助手：{response}")

----------------对话1----------------
用户：什么是人工智能。
助手：我认为未来的人工智能发展方向包括更强大的自主性、更加灵活的适应能力、更好的伦理道德规范以及更多的应用场景。同时，我们也需要关注AI对社会的影响，并采取相应的措施来应对可能出现的问题。例如，我们需要确保数据的安全性和隐私保护，同时也需要加强AI伦理教育，让公众了解并接受这种技术带来的影响。总的来说，人工智能将是一个不断发展的领域，需要我们不断地探索和创新。
----------------对话2----------------
用户：列举它的应用场景。
助手：人工智能的应用场景非常广泛，其中最常见的是医疗保健，如通过分析大量的医学影像数据进行疾病诊断；在金融行业，可以用于风险评估和投资决策；在交通行业中，可以通过预测车辆行驶路径来优化公共交通系统等。此外，它还被应用于制造业、农业、零售业等多个领域，帮助人们提高效率和生活质量。当然，随着技术的发展，人工智能还在不断拓展其应用范围，比如智能家居、虚拟现实等。
----------------对话3----------------
用户：使用人工智能需要注意什么。
助手：首先，我们需要重视数据安全和个人隐私保护。其次，我们需要遵守相关的法律法规，以避免可能的风险和后果。最后，我们需要持续学习和更新自己的知识和技术，以跟上人工智能发展的步伐。总的来说，人工智能是一项充满机遇和挑战的技术，只有做好充分准备才能更好地发挥其潜力。


在多轮对话中，对话历史的维护是关键。通过将多轮对话拼接作为模型的输入，模型能够根据之前的对话内容生成更连贯的回答。你可以根据实际需求调整对话历史的长度，以平衡模型的推理速度和对话连贯性。

以上内容详细介绍了如何使用开源大语言模型进行单轮和多轮对话。在实际应用中，你可能需要根据具体需求进一步优化和扩展这些示例。

## 2.7 小结

通过本章内容，我们简单学习了生成式大语言模型的原理、使用及特点。首先，我们介绍了基于Transformer架构的预训练思想，列举了主流模型。接着，阐述了文本生成原理，包括词汇表、Token、自回归预测等概念，探讨了预训练和推理带给语言模型的创意性、随机性及局限性。然后，基于Hugging Face Transformers，我们演示了加载开源模型（如Qwen2.5-Insturct）进行文本生成的方法，介绍了影响生成结果的参数和技巧。最后，提供了构建单轮问答助手和多轮聊天机器人的代码案例，帮助读者掌握对话系统的简单开发。


## 2.8 参考文献

[1] Qwen2.5: A Party of Foundation Models

[2] Attention is all you need

[3] A Survey of Large Language Models

[4] Transformers: State-of-the-Art Natural Language Processing

[5] Improving Language Understanding by Generative Pre-training

[6] Language Models are Unsupervised Multitask Learners

[7] Language Models are Few-Shot Learners


## 2.9 课后习题

下面让我们来进行几道简单的练习，巩固一下本章所学的知识吧！

### 习题 1：关于温度参数
**问题**：在生成式大语言模型中，温度参数（Temperature）的作用是什么？
A. 控制生成文本的长度  
B. 控制生成文本的多样性  
C. 控制生成文本的速度  
D. 控制生成文本的格式  

**答案**：B. 控制生成文本的多样性

### 习题 2：关于采样策略
**问题**：以下哪种采样策略会从概率最高的 k 个词中随机选择下一个词？
A. 贪心采样（Greedy Sampling）  
B. Top-k 采样  
C. Top-p 采样（Nucleus Sampling）  
D. 随机采样  

**答案**：B. Top-k 采样

### 习题 3：解释 Top-p 采样
**问题**：解释 Top-p 采样（Nucleus Sampling）的工作原理及其优点。

**解答**：
Top-p 采样，也称为 Nucleus Sampling，是一种改进的采样方法，它从概率累积达到 p 的最小词集中随机选择下一个词。具体步骤如下：
1. 计算每个词的生成概率。
2. 按概率从高到低排序。
3. 累积概率，直到总和达到或超过设定的阈值 p。
4. 从这个累积概率达到 p 的最小词集中随机选择下一个词。

优点：
- 动态调整采样范围，避免了固定 k 值的 Top-k 采样可能带来的问题。
- 能够更好地平衡生成文本的多样性和稳定性。
- 适应不同的生成需求，生成结果更自然。

### 习题 4：如何调整参数以生成创意文本
**问题**：在生成式大语言模型中，如何通过调整参数来生成更具创意的文本？

**解答**：
为了生成更具创意的文本，可以调整以下参数：
- **提高温度参数（Temperature）**：设置较高的温度值（如 1.0 或更高），增加生成结果的多样性。
- **使用 Top-p 采样**：设置较低的 p 值（如 0.9），从概率累积达到 p 的最小词集中随机选择下一个词，增加生成结果的随机性。
- **避免使用贪心采样**：贪心采样总是选择概率最高的词，生成结果较为稳定但缺乏创意。

### 习题 5：实现一个简单的多轮对话系统
**问题**：自学`Transformers`的`.generate`函数，使用 Qwen2.5-Insturct 模型，实现一个简单的多轮对话系统。要求能够维护对话历史，并生成连贯的对话。

**解答**：


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# 加载模型和分词器
model_name = "Qwen2.5-0.5B-Insturct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

class MultiTurnChatbot:
    def __init__(self):
        self.history = []

    def generate_response(self, user_input):
        # 构建包含历史的提示
        prompt = "对话历史：\n"
        for turn in self.history:
            prompt += f"用户：{turn['user']}\n助手：{turn['assistant']}\n"
        prompt += f"用户：{user_input}\n助手："

        # 编码输入文本
        inputs = tokenizer(prompt, return_tensors="pt")

        # 生成回答
        outputs = model.generate(
            **inputs,
            max_length=500,
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id
        )

        # 解码生成的文本
        response = tokenizer.decode(outputs[0], skip_special_tokens=True).split("助手：")[-1]

        # 保存当前对话到历史
        self.history.append({"user": user_input, "assistant": response})
        return response

# 创建聊天机器人实例
chatbot = MultiTurnChatbot()

print("欢迎使用聊天机器人！输入 '退出' 停止对话。")

while True:
    user_input = input("\n用户：")
    if user_input.lower() == "退出":
        print("聊天结束，再见！")
        break
    response = chatbot.generate_response(user_input)
    print(f"助手：{response}")